# Keras Implementation Tutorial

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense

In [4]:
import numpy as np
# fix random seed for reproducibility
#seed = 7
#numpy.random.seed(seed)

## 0. The Data

In this tutorial, we are going to study what attributes infuence Pima Indians to have diabetes. The dataset is from [UIC Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Pima+Indians+Diabetes). All samples here are females at least 21 years old of Pima Indian heritage. 

This is a binary classification problem, and Class = 1 means patients having diabetes. The number of instances (samples) is 768 and the number of attributes is 8, which are **(1) Number of times pregnant** 
**(2) Plasma glucose concentration a 2 hours in an oral glucose tolerance test**,
**(3) Diastolic blood pressure (mm Hg)**,
**(4) Triceps skin fold thickness (mm)**,
**(5) 2-Hour serum insulin (mu U/ml)**,
**(6) Body mass index (weight in kg/(height in m)^2)**,
**(7) Diabetes pedigree function**,
**(8) Age (years)**.

In [7]:
import pandas as pd
dataset = pd.read_csv("pima-indians-diabetes.csv")
dataset.columns = ['num_pregnant','glucose_concentration','blood_pressure','skin_fold_thickness','serum_insulin','body_mass_index','diabetes_pedigree','age','class']
dataset.head()

,num_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,body_mass_index,diabetes_pedigree,age,class
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [8]:
dataset.shape

(767, 9)

### Split dataset in training and test datasets

In [9]:
X = dataset.iloc[:,:8]
Y = dataset.iloc[:,8]

In [11]:
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn import metrics, cross_validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
type(X_train), type(Y_train)

(pandas.core.frame.DataFrame, pandas.core.series.Series)

In [12]:
X_train.shape, Y_train.shape

((536, 8), (536,))

In [13]:
X_test.shape, Y_test.shape

((231, 8), (231,))

Note that different from sklearn, the **input data format in Keras needs to be np.array**. Thus here we have to convert the pandas frame to numpy array: 

In [17]:
X_train_arr = X_train.as_matrix()
Y_train_arr = Y_train.as_matrix()

In [18]:
X_test_arr = X_test.as_matrix()
Y_test_arr = Y_test.as_matrix()

In [79]:
type(X_train_arr), type(Y_train_arr)

(numpy.ndarray, numpy.ndarray)

## 1. Define the Neural Network

### (a) model I: 3-layer network, unit numbers are 12-8-1, relu activation function

First we consider one-hidden layer, so the neural network architecture is **input-hidden-output**:

In [14]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

The input dimension is 8, since there are 8 attributes in the data. The above neural network structure shows that the input layer has 12 neurons (units), hidden one has 8 neurons. The output always has 1 neuron. Since this is a binary classification problem, we always implement **sigmoid** activation function in the output layer to indicate probability to have diabetes. Once the probability > 0.5, the model will predict the patient of patients having diabetes, i.e. $\hat{y}_{pred} =1$, otherwise $\hat{y}_{pred}=0$.

For input layer and hidden layers, we can still choose **sigmoid** or **[relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)** as the activation function. **relu** activation function is popular in deep learning, in particular for convolutional neural networks. For now we use **relu**. Later we will revisit the same network but using **sigmoid** activation and compare the performances.


We use **logarithmic loss** as the loss (cost) function in stochastic graident descent. In Keras, for a **binary** classification problem it is **binary_crossentropy**. For multiclass classification problems, the loss function can be **categorical_crossentropy** or **mse**. To train the neural networks, we implement gradient descent to minimize the loss function. In Keras, we can choose **Adam** or **SGD** for gradient descent. **Adam** is an efficient gradient descent algorithm. Learning more about the Adam optimization algorithm, please refer to the paper [“Adam: A Method for Stochastic Optimization“](https://arxiv.org/abs/1412.6980). The metric used here is "accuracy". 

In [15]:
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Training model

We first set up number of epoches as **nb_epoch=150**, and the size of each mini-batch as **batch_size=10**. Keep in mind that different from sklearn, the input data format in Keras need to be **numpy.array** (in sklearn input data can be dataframe).

In [19]:
model.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=10, verbose=0)

#### Evaluate the model

In [20]:
scores = model.evaluate(X_train_arr, Y_train_arr)
print('  Loss: ', scores[0], ' , acc:', scores[1])

352/536 [==================>...........] - ETA: 0s  Loss:  0.510310189493  , acc: 0.753731342394


In [21]:
scores = model.evaluate(X_test_arr, Y_test_arr)
print('  Loss: ', scores[0], ' , acc:', scores[1])

231/231 [==============================] - 0s     
  Loss:  0.543567587545  , acc: 0.740259740389


The **acc** is the accuracy defined by the ratio we have right prediction. This is the same as doing:

In [23]:
predictions = model.predict(X_test_arr)
rounded = [round(x[0]) for x in predictions]
a = np.dot(rounded-Y_test_arr, rounded-Y_test_arr)
print(1.0-a/len(rounded))

0.74025974026


We can further compute the probability of each Pima Indian patient having diabetes as

In [24]:
print (model.predict_proba(X_test_arr)[:10])

192/231 [=======================>......] - ETA: 0s[[ 0.58446217]
 [ 0.40370643]
 [ 0.16911635]
 [ 0.2323039 ]
 [ 0.0529535 ]
 [ 0.20644023]
 [ 0.12511159]
 [ 0.08647307]
 [ 0.23116459]
 [ 0.42215642]]


#### Double mini-batch size

Now we can double the mini-batch size and train the model again:

In [25]:
model.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=20, verbose=0)
scores = model.evaluate(X_test_arr, Y_test_arr)
print('  Loss: ', scores[0], ' , acc:', scores[1])

 32/231 [===>..........................] - ETA: 0s  Loss:  0.520820880865  , acc: 0.753246754021


#### Implement SGD

Next we try SGD as the gradient descent method. We can see the performance is worse than that using **adam** algorithm:

In [26]:
from keras.optimizers import SGD
#model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [27]:
model.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=10, verbose=0)
scores = model.evaluate(X_test_arr, Y_test_arr);
print('  Loss: ', scores[0], ' , acc:', scores[1])

 32/231 [===>..........................] - ETA: 0s  Loss:  0.586787244974  , acc: 0.653679654454


### (b) model II: 3-layer network, unit numbers are 40-30-1, relu activation function

In [28]:
model2 = Sequential()
model2.add(Dense(40, input_dim=8, init='uniform', activation='relu'))
model2.add(Dense(30, init='uniform', activation='relu'))
model2.add(Dense(1, init='uniform', activation='sigmoid'))

In [29]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model2.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=10, verbose=0)
scores = model2.evaluate(X_train_arr, Y_train_arr);
print('  Loss: ', scores[0], ' , acc:', scores[1])

352/536 [==================>...........] - ETA: 0s  Loss:  0.443567765738  , acc: 0.776119402985


In [31]:
scores = model2.evaluate(X_test_arr, Y_test_arr);
print('  Loss: ', scores[0], ' , acc:', scores[1])

224/231 [============================>.] - ETA: 0s  Loss:  0.594045774761  , acc: 0.740259740389


We can see this shows using more units will train more accurate models, but has slightly lower accuracy on test dataset. Therefore using too many neurons will result in overfitting!

### (c) model III: 3-layer network, unit numbers are 12-8-1, sigmoid activation function

Now let's do deep learning by considering **sigmoid** as activation function. The accuracy is slightly better than using **relu**.

In [32]:
model3 = Sequential()
model3.add(Dense(12, input_dim=8, init='uniform', activation='sigmoid'))
model3.add(Dense(8, init='uniform', activation='sigmoid'))
model3.add(Dense(1, init='uniform', activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=10, verbose=0)
scores = model3.evaluate(X_test_arr, Y_test_arr);
print('  Loss: ', scores[0], ' , acc:', scores[1])

 32/231 [===>..........................] - ETA: 0s  Loss:  0.541187678839  , acc: 0.701298702073


### (d) model IV: 4-layer network, unit numbers are 12-8-8-1, relu activation function

Here we use two hidden layers, the network structure is **input-hidden-hidden-output**, and numbers of units on the both hidden layers are 8.

In [34]:
model4 = Sequential()
model4.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model4.add(Dense(8, init='uniform', activation='relu'))
model4.add(Dense(8, init='uniform', activation='relu'))
model4.add(Dense(1, init='uniform', activation='sigmoid'))
model4.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model4.fit(X_train_arr, Y_train_arr, nb_epoch=150, batch_size=10, verbose=0)
scores = model4.evaluate(X_test_arr, Y_test_arr);
print('  Loss: ', scores[0], ' , acc:', scores[1])

 32/231 [===>..........................] - ETA: 0s  Loss:  0.565378722174  , acc: 0.72294372346


Compared with the previous results using three layers, the four-layer network does not significantly improve model accuracy (compared to model II).

## 2. Prediction

In [41]:
predictions = model.predict_classes(X_test_arr)
predictions.shape
predictions[:10]

128/231 [===============>..............] - ETA: 0s

array([[1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int32)

In [37]:
rounded = [round(x[0]) for x in predictions]
print (rounded[:10])

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [39]:
predictions2 = model.predict_proba(X_test_arr)
predictions2[:10]

231/231 [==============================] - 0s     


array([[ 0.87228608],
       [ 0.55333531],
       [ 0.2439429 ],
       [ 0.30015633],
       [ 0.14511333],
       [ 0.28503677],
       [ 0.14051481],
       [ 0.11668609],
       [ 0.35142043],
       [ 0.39413071]], dtype=float32)

## 3. Compare to Logistic-Regression, SVM and Random-Forest models

Next we try to compare the neural network classification with other machine learning models, such as logistic-regression, support-vector-machine and random-forest models. We train the models with 10-fold cross-validation set,  perform grid search to find the best model and the hyperparameters, and then compute the model accuracy using the test dataset. We will see that in the current small dataset case, the logistic regression, SVM and random forest also show similar accuracy as the neural network did.

### (a) Logistic regression

In [43]:
from sklearn import linear_model
best_logreg_model = None
max_score = -1
best_reg = -1
for regularization_param in [0.01, 0.1, 1, 2, 10, 100, 500, 1000, 5000]:
    logreg = linear_model.LogisticRegression('l2', C=regularization_param)
    cv_score = cross_val_score(logreg, X_train, Y_train, cv=10)
    print (regularization_param, np.mean(cv_score))
    if np.mean(cv_score) > max_score:
        max_score = np.mean(cv_score)
        best_logreg_model = logreg
        best_reg = regularization_param
        
best_logreg_model.fit(X_train, Y_train)
print ('best reg =', best_reg)
print ('accuracy = ', best_logreg_model.score(X_test, Y_test))

0.01 0.673559494314
0.1 0.71091099676
1 0.765182644051
2 0.772557651992
10 0.778219299917
100 0.776332507465
500 0.778219299917
1000 0.776332507465
5000 0.778219299917
best reg = 500
accuracy =  0.761904761905


### (b) Support-vector-machine

In [121]:
from sklearn import svm
best_svm_model = None
best_Cs = -1
max_score = -1
for Cs in [0.1, 0.2, 0.5, 0.7, 1.0, 2.0, 5.0]:
    svc = svm.SVC(kernel='linear', C=Cs, probability=True)
    ### usually not suggest to do cv in SVM since it costs time a lot, even an iteration
    cv_score = cross_val_score(svc, X_train, Y_train, cv=5) 
    print (Cs, np.mean(cv_score))
    if np.mean(cv_score) > max_score:
        max_score = np.mean(cv_score)
        best_svm_model = svc
        best_Cs = Cs

best_svm_model.fit(X_train, Y_train)
print ('best C =', best_Cs)
print ('accuracy = ', best_svm_model.score(X_test, Y_test))

0.1 0.777933541018
0.2 0.776064382139
0.5 0.77978539287
0.7 0.776047075112
1.0 0.777916233991
2.0 0.776047075112
5.0 0.774177916234
best C = 0.5
accuracy:  0.753246753247


In [126]:
print ('MSE = ', best_svm_model.score(X_test, Y_test))

MSE =  0.753246753247


### (c) Random forest

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import grid_search
rf = RandomForestClassifier()
parameters = {'n_estimators': [4,6,8],'max_depth':[5,10,15],'min_samples_leaf':[10,20]}
model_cv_grid = grid_search.GridSearchCV(rf, parameters, scoring='roc_auc', verbose=0, n_jobs=-1, cv=10)
model_cv_grid.fit(X_train,Y_train)
best_rf_model = model_cv_grid.best_estimator_

/Users/hhhung/anaconda/envs/tensorflow/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [46]:
best_rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=10,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=6, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [47]:
print ('accuracy = ', best_rf_model.score(X_test, Y_test))

accuracy =  0.744588744589


In [48]:
importance = best_rf_model.feature_importances_
attribute = X.columns

v = sorted(range(len(importance)), key=lambda k: importance[k], reverse=True)
sorted_importance = [importance[i] for i in v]
sorted_attribute = [attribute[i] for i in v]

df_importance = pd.DataFrame({'variable': sorted_attribute, 'importance' : sorted_importance})
df_importance.sort_index().head(5)

,importance,variable
0,0.323314,glucose_concentration
1,0.190571,body_mass_index
2,0.125323,diabetes_pedigree
3,0.115889,age
4,0.094110,serum_insulin


## Reference

* 1. [Develop Your First Neural Network in Python With Keras Step-By-Step](http://machinelearningmastery.com/tutorial-first-neural-network-python-keras/)